<a href="https://colab.research.google.com/github/OndwelaT/OndwelaT.github.io/blob/main/Functional_twitter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install requests-oauthlib

In [ ]:
import json
import os
import requests
from glob import glob as glob
import shutil
from google.colab import files
import re
import pandas as pd

In [ ]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAFLqvAEAAAAAgQzczeKNfPrpbeQXv24BD%2F4ARLM%3DzbFNZWVYczIe5h8HZpDdd924QqrvIC0v5gCjFn8D8Y8QazqGa7"
search_url = "https://api.twitter.com/2/tweets/search/recent"

In [ ]:
#query_params = {
 #   'query': 'water supply KZN -is:retweet',
 #   'tweet.fields': 'author_id',
 #   'max_results': 100}



In [ ]:
def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

In [ ]:
def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
def save(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

In [ ]:
#def main():
 #   json_response = connect_to_endpoint(search_url, query_params)
 #   print(json.dumps(json_response, indent=4, sort_keys=True))

 #   save(json_response, 'twitter_data.json')
 #   print("Data saved to twitter_data.json")

In [ ]:
def main():
    max_results = 1000
    tweets = []
    next_token = None

    while len(tweets) < max_results:
        query_params = {
            'query': 'Water quality  South Africa -is:retweet',
            'tweet.fields': 'author_id',
            'max_results': 100
        }

        if next_token:
            query_params['next_token'] = next_token

        json_response = connect_to_endpoint(search_url, query_params)
        tweets.extend(json_response.get('data', []))

        if 'meta' in json_response and 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
        else:
            break


    print(f"Obtained {len(tweets)} tweets")

    save({"data": tweets}, 'twitter_data.json')
    print("Data saved to twitter_data.json")

In [ ]:
if __name__ == "__main__":
    main()

200
Obtained 2 tweets
Data saved to twitter_data.json


loading the datasets

In [ ]:
directory_path = "/content/twitter_data_directory"
os.makedirs(directory_path, exist_ok = True)

if os.path.exists(directory_path):
    print("Directory created.")
else:
    print("Directory could not be created.")

Directory created.


In [ ]:
#files to be stored saved into a list
files = ["/content/twitter_data.json"]

twitter_data = []
for file_paths in files:
  with open(file_paths, "r") as file:
    data = json.load(file)
    twitter_data.append(data)

convert json files into excel

In [ ]:
merged_data_file = "twitter_merged_data.json"

with open(merged_data_file, "w") as file:
  json.dump(twitter_data, file, indent = 4)

print("JSON files have been merged")

JSON files have been merged


In [ ]:
#load data
with open("twitter_merged_data.json") as file:
  uncleaned_data = json.load(file)
  print("file loaded")

file loaded


In [ ]:
json_data = uncleaned_data
json_data

[{'data': [{'id': '1837069194917232946',
    'edit_history_tweet_ids': ['1837069194917232946'],
    'author_id': '1411990690779865089',
    'text': 'South Africa is a water scarce country. In other words it has an excess of demand over available supply. It also has low water security – the ability to ensure sustainable access to good quality water. #AfricanInsider \nhttps://t.co/pqTNEj2JhD'},
   {'id': '1837008790333313366',
    'edit_history_tweet_ids': ['1837008790333313366'],
    'author_id': '2906219303',
    'text': '[South Africa] The lack of effective management at sewage plants has led to deteriorating water quality and increased pressure on municipalities. At Harmony, we are committed to mining with purpose, and creating value for all stakeholders. \n#MiningWithPurpose https://t.co/gH5iQdHbck'}]}]

In [ ]:
df_columns = ["author_id", "text", "id","edit_history_tweet_ids"]

extract_data = []

for inner_dict in json_data[0]["data"]:
    extract_data.append([inner_dict[column] for column in df_columns])

print(extract_data)

[['1411990690779865089', 'South Africa is a water scarce country. In other words it has an excess of demand over available supply. It also has low water security – the ability to ensure sustainable access to good quality water. #AfricanInsider \nhttps://t.co/pqTNEj2JhD', '1837069194917232946', ['1837069194917232946']], ['2906219303', '[South Africa] The lack of effective management at sewage plants has led to deteriorating water quality and increased pressure on municipalities. At Harmony, we are committed to mining with purpose, and creating value for all stakeholders. \n#MiningWithPurpose https://t.co/gH5iQdHbck', '1837008790333313366', ['1837008790333313366']]]


In [ ]:
#sort out data befor preprocessing
df = pd.DataFrame(extract_data, columns = df_columns)

excel_file_path = "/content/data_to_sort.xlsx"

df.to_excel(excel_file_path, index = False)

